In [ ]:
### Data pulled from .csv files
'''
https://projects.fivethirtyeight.com/congress-model-2018/house_national_forecast.csv

https://projects.fivethirtyeight.com/congress-model-2018/house_district_forecast.csv

https://projects.fivethirtyeight.com/congress-model-2018/senate_national_forecast.csv

https://projects.fivethirtyeight.com/congress-model-2018/senate_seat_forecast.csv '''

In [1]:
import pandas as pd
import sqlite3

In [2]:
def user_choice():
    n = 0
    while n == 0:
        choice = input("Which data base would you like up date? Enter \n \
              '1' For the House forecast \n \
              '2' For the Senate forecast\n")
        if choice in str([x+1 for x in range(2)]):
            n=1
            return choice
        else:
            print('Wrong input :(')
            continue        

In [3]:
choice = user_choice()

Which data base would you like up date? Enter 
               '1' For the House forecast 
               '2' For the Senate forecast
2


In [5]:
def pull_data(choice):
    if choice == "1":    
        poll_data = pd.read_csv('https://projects.fivethirtyeight.com/congress-model-2018/house_national_forecast.csv')
        elect_type = 'house'
    elif choice == "2":
        poll_data = pd.read_csv('https://projects.fivethirtyeight.com/congress-model-2018/senate_national_forecast.csv')
        elect_type = 'senate'
    else:
        print('The programmer messed up. Sorry. Try re-running.')
        return
    return poll_data, elect_type      
        

In [21]:
senate_data,elect_type = pull_data('2')

In [31]:
def clean(data):
    data = data.drop(['p10_seats','p90_seats','margin','p10_margin','p90_margin'],axis = 1)
    data_D = data.query("'D' == party")
    data_R = data.query("'R' == party")
    data = data_D.merge(data_R, how='inner', on=['forecastdate','state','model'])
    data = data.rename(columns = {'win_probability_x':'d_win_prob','mean_seats_x':'d_mean_seats','median_seats_x':'d_med_seats', \
                             'win_probability_y':'r_win_prob','mean_seats_y':'r_mean_seats','median_seats_y':'r_med_seats'})
    data = data.drop(['state','party_x','party_y'],axis = 1)
    return data
    

In [17]:
# 'data' is for the dataframe, 'elect_type' indicates whether it is the Senate or House Forecast
def add_to_db(data,elect_type): 
    elect_type_db = str(elect_type) + '.db'
    con = sqlite3.connect(elect_type_db)
    try:
        data_db = pd.read_sql('select * from data', con)
    except:
        pd.DataFrame.to_sql(data,con=con,name='data')
        con.close()
        return
    data_db = data_db.append(data,sort=False)
    data_db_push = data_db.drop_duplicates(subset='TimeStamp')
    pd.DataFrame.to_sql(data_db_push,con=con,name='data')
    con.close()
    return

In [34]:
def master():
    choice = user_choice()
    data,elect_type = pull_data(choice)
    data = clean(data)
    add_to_db(data,elect_type)
    return print(elect_type,'data updated')